In [66]:
import matplotlib.pyplot as plt
import numpy as np
import gzip as gz

In [67]:
def pca(data,k_features):
    # check 0 < k_features <= number of features
    if k_features > 0 and k_features <= data.shape[1]:

        # center the data and calculate the covariance matrix (sigma)
        sigma = np.corrcoef(data.T)

        # get the eigenvectors of sigma
        eigen_vecs, _, _ = np.linalg.svd(sigma)
        
        # create an empty matrix to hold dimensionally reduced data
        reduced_data = np.empty((data.shape[0], k_features))

        # for each observation x, project x onto eigenvectors
        for observation_idx in range(data.shape[0]):
            reduced_data[observation_idx] = np.dot(eigen_vecs[:,:k_features].T, data[observation_idx,:][:,np.newaxis])[:,np.newaxis].T
            print reduced_data.shape

        # return dimensionally reduced data
        return reduced_data
    
    # print error message
    print ('ERROR: 0 < k_features < %i') % data.shape[1]   

In [68]:
def get_cases(data,cases):
    logical_array = np.logical_or.reduce([data[:,0] == case for case in cases])
    return data[logical_array]

In [65]:
with gz.open("../Data/zip.train.gz") as f:
    train_data = np.loadtxt(f)

#train_data = get_cases(train_data,(7,9))

X_train = train_data[:,1:]
y_train = train_data[:,0]

X_train = pca(train_data[:,1:],train_data.shape[1] - 1)


print X_train

(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(7291, 256)
(729

(147, 256)
